Tools

Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:

    1.A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
    2.A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots can talk. Let me start by recalling what I know about parrots. They\'re known for mimicking human speech, right? But why do they do that? Is it just for communication, or is there more to it?\n\nFirst, maybe I should think about their biology. Parrots have a syrinx instead of a larynx, which is the vocal organ in birds. I remember that the syrinx allows them to produce a wide range of sounds. But how does that relate to talking? They can imitate sounds they hear, like people talking. So maybe it\'s about learning through imitation.\n\nThen there\'s the social aspect. Parrots are social animals and often live in groups in the wild. If they can mimic sounds, maybe they use it to communicate with each other. But when they\'re around humans, they mimic human speech. Could it be a way to bond or to fit into their environment? Like, if they\'re in a human household, they learn to copy the speech to interact with their huma

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there\'s no typo. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': 'ntpp227z9', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 154, 'total_tokens': 247, 'completion_time': 0.143151986, 'completion_tokens_details': {'reasoning_tokens': 69}, 'prompt_time': 0.005997648, 'prompt_tokens_details': None, 'queue_time': 0.057378722, 'total_time': 0.149149634}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_

Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is currently sunny.


In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. Let me check the tools available. There\'s a function called get_weather that takes a location parameter. The required parameter is location, which should be a string. Since the user specified Boston, I need to call get_weather with location set to "Boston". I\'ll make sure the JSON is correctly formatted with the name of the function and the arguments as specified.\n', 'tool_calls': [{'id': 'rd1bfqhn9', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 153, 'total_tokens': 261, 'completion_time': 0.164419841, 'completion_tokens_details': {'reasoning_tokens': 84}, 'prompt_time': 0.006898434, 'prompt_tokens_details': None, 'queue_time': 0.057129966, 'total_time': 0.171318275}, 'model_nam